In [1]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

# 在agent与tool之间共享记忆

- 自定义一个工具用来LLMChain来总结内容
- 使用readonlymemory来共享记忆
- 观察共享与不共享的区别
<hr>

In [2]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, MessagesPlaceholder
from langchain.utilities import SerpAPIWrapper

# 创建llm
llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo-instruct",
)

/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


创建一条链来总结对话

In [3]:
template = """以下是一段AI机器人和人类的对话:
{chat_history}
根据输入和上面的对话记录写一份对话总结.
输入: {input}"""

prompt = PromptTemplate(
    input_variables=["input", "chat_history"],
    template=template,
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)
readonly_memory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=readonly_memory,
)


构建工具

In [4]:
import os

os.environ["SERPAPI_API_KEY"] = "32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719"
#搜索工具
search = SerpAPIWrapper()


#总结工具
def summary_chain_fun(history):
    print("\n==============总结链开始运行==============")
    print("输入历史: ", history)
    summary_chain.run(history)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="当需要了解实时的信息或者你不知道的事时候可以使用搜索工具",
    ),
    Tool(
        name="Summary",
        func=summary_chain_fun,
        description="当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用",
    ),
]
print(tools)

[Tool(name='Search', description='当需要了解实时的信息或者你不知道的事时候可以使用搜索工具', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='32109af1224c43d11768866970c41e4e681e03bf220c7bd2b9a16af817431719', aiosession=None)>), Tool(name='Summary', description='当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用', func=<function summary_chain_fun at 0x111230d60>)]


创建记忆组件

In [5]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

创建agent

In [6]:
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    memory=memory,
)

/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [8]:
print(agent_chain.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: 当需要了解实时的信息或者你不知道的事时候可以使用搜索工具
Summary: 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


重构prompt,加入历史记录

In [10]:
prefix = """Answer the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!
{chat_history}
Question: {input}
Thought:{agent_scratchpad}
"""

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={
        "prefix":prefix,
        "suffix":suffix,
        "chat_history":MessagesPlaceholder("chat_history"),
        "input":MessagesPlaceholder("input"),
        "agent_scratchpad":MessagesPlaceholder("agent_scratchpad"),
    },
    memory=memory,
)
print(agent_chain.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: 当需要了解实时的信息或者你不知道的事时候可以使用搜索工具
Summary: 当你被要求总结一段对话的时候可以使用这个工具，工具输入必须为字符串，只在必要时使用

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Summary]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
{chat_history}
Question: {input}
Thought:{agent_scratchpad}



In [11]:
agent_chain.invoke(input="美国第45任总统是谁?")



> Entering new AgentExecutor chain...
Action: Search
Action Input: 美国第45任总统
Observation: 唐納·約翰·川普（英語：Donald John Trump [註4]；1946年6月14日—），美國政治人物，第45任美国总统。 從政前為企业家、媒體名人。 川普出生并成长于纽约州紐約市皇后區，为特朗普集團前任董事長兼总裁及特朗普娱乐公司的創辦人，在全世界经营房地产、賭場和酒店 [註5]。
Thought:Action: Summary
Action Input: 唐納·約翰·川普（英語：Donald John Trump [註4]；1946年6月14日—）
==============总结链开始运行==============
输入历史:  唐納·約翰·川普（英語：Donald John Trump [註4]；1946年6月14日—）


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话:
[]
根据输入和上面的对话记录写一份对话总结.
输入: 唐納·約翰·川普（英語：Donald John Trump [註4]；1946年6月14日—）


/Users/jianghui/PycharmProjects/ai_agent_study/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



> Finished chain.

Observation: None
Thought:Action: Summary
Action Input: 川普出生并成长于纽约州紐約市皇后區，为特朗普集團前任董事長兼总裁及特朗普娱乐公司的創辦人，在全世界经营房地产、賭場和酒店 [註5]
==============总结链开始运行==============
输入历史:  川普出生并成长于纽约州紐約市皇后區，为特朗普集團前任董事長兼总裁及特朗普娱乐公司的創辦人，在全世界经营房地产、賭場和酒店 [註5]


> Entering new LLMChain chain...
Prompt after formatting:
以下是一段AI机器人和人类的对话:
[]
根据输入和上面的对话记录写一份对话总结.
输入: 川普出生并成长于纽约州紐約市皇后區，为特朗普集團前任董事長兼总裁及特朗普娱乐公司的創辦人，在全世界经营房地产、賭場和酒店 [註5]

> Finished chain.

Observation: None
Thought:Final Answer: 唐納·約翰·川普（英語：Donald John Trump [註4]；1946年6月14日—），美國政治人物，第45任美国总统。

> Finished chain.


{'input': '美国第45任总统是谁?',
 'chat_history': [HumanMessage(content='美国第45任总统是谁?'),
  AIMessage(content='唐納·約翰·川普（英語：Donald John Trump [註4]；1946年6月14日—），美國政治人物，第45任美国总统。')],
 'output': '唐納·約翰·川普（英語：Donald John Trump [註4]；1946年6月14日—），美國政治人物，第45任美国总统。'}

In [12]:
agent_chain.invoke(input="他的夫人叫什么名字?")



> Entering new AgentExecutor chain...
Action: Search
Action Input: 唐納·約翰·川普
Observation: ['Donald John Trump is an American politician, media personality, and businessman who served as the 45th president of the United States from 2017 to 2021. Trump received a Bachelor of Science in economics from the University of Pennsylvania in 1968, and his father named him president of his real estate business in 1971.', 'Donald Trump (唐納·川普) type: 45th U.S. President.', 'Donald Trump (唐納·川普) entity_type: people, people.', 'Donald Trump (唐納·川普) main_tab_text: Overview.', 'Donald Trump (唐納·川普) kgmid: /m/0cqt90.', 'Donald Trump (唐納·川普) born: June 14, 1946 (age 77 years), Jamaica Hospital Medical Center, New York, NY.', 'Donald Trump (唐納·川普) party: Republican Party.', 'Donald Trump (唐納·川普) spouse: Melania Trump (m. 2005), Marla Maples (m. 1993–1999), Ivana Trump (m. 1977–1990).', 'Donald Trump (唐納·川普) net_worth: 4.4 billion USD (2024).', 'Donald Trump (唐納·川普) vice_president: Mike Pence (2017–2021).

{'input': '他的夫人叫什么名字?',
 'chat_history': [HumanMessage(content='美国第45任总统是谁?'),
  AIMessage(content='唐納·約翰·川普（英語：Donald John Trump [註4]；1946年6月14日—），美國政治人物，第45任美国总统。'),
  HumanMessage(content='他的夫人叫什么名字?'),
  AIMessage(content='唐納·約翰·川普（Donald John Trump）')],
 'output': '唐納·約翰·川普（Donald John Trump）'}